In [1]:
import tensorflow as tf
import numpy as np 

In [2]:
from fast_scnn import fastSCNN, fastSCNN_edition2
from network import network_big

In [3]:
input_shape=(256,192, 3)
model = fastSCNN(input_shape, train=False)
# model = fastSCNN_edition2(input_shape)
# model = network_big(input_shape, train=False)

In [4]:
model.output

<tf.Tensor 'output/Identity:0' shape=(None, 256, 256, 1) dtype=float32>

In [5]:
model_path = "./trained_models/20200824/5_0.13.h5"
model.load_weights(model_path, by_name=True)

In [6]:
# model.save(model_path.replace(".h5", "_withGraph.h5"))

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 8)  224         input_1[0][0]                    
__________________________________________________________________________________________________
p_re_lu (PReLU)                 (None, 256, 256, 8)  8           conv2d[0][0]                     
__________________________________________________________________________________________________
separable_conv2d (SeparableConv (None, 256, 256, 8)  144         p_re_lu[0][0]                    
______________________________________________________________________________________________

**실행 시간 체크**

In [7]:
x = np.random.random(size=(1,) + input_shape)

In [8]:
%%timeit # 기존 모델의 실행시간
pred = model.predict(x)

113 ms ± 4.69 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit # 현재 기준 
pred = model.predict(x)

43.9 ms ± 1.69 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit  # 추후 시도 할 것의 기준
pred = model.predict(x)

42.4 ms ± 3.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%timeit  # 추후 시도 할 것의 기준
pred = model.predict(x)

40.3 ms ± 663 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision

In [9]:
model.output

<tf.Tensor 'bilinear_interpolation_5/Identity:0' shape=(None, 256, 192, 1) dtype=float32>

### To Convert TFJS model

In [10]:
# save tf-saved model
tf_saved = model_path.replace(".h5", "_InTF_savedModel")
tf.saved_model.save(model, tf_saved)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./trained_models/widerScale/2862-0.05_InTF_savedModel/assets


In [11]:
import tensorflowjs
tensorflowjs.converters.convert_tf_saved_model(tf_saved, model_path.replace(".h5", "_tfjs_models"))

Writing weight file ./trained_models/widerScale/2862-0.05_tfjs_models/model.json...
